In [4]:
%pip install pandas
%pip install torchvision 



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: C:\Users\patel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 1.3/1.3 MB 8.9 MB/s eta 0:00:00
     -------------------------------------- 192.3/192.3 MB 7.1 MB/s eta 0:00:00
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
     -------------------------------------- 166.4/166.4 kB 9.8 MB/s eta 0:00:00
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: C:\Users\patel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

In [7]:
bread = np.load("combinedarr/bread_combinedarr.npy",allow_pickle=True)
night = np.load("combinedarr/night_combinedarr.npy",allow_pickle=True)
picnic = np.load("combinedarr/picnic_combinedarr.npy",allow_pickle=True)

In [8]:
print(len(bread))
print(len(night))
print(len(picnic))

1444
1024
729


In [9]:
combined = np.concatenate((bread, night, picnic))
len(combined)

3197

In [15]:
data = [c[0] for c in (combined[:,0:1])]
label = combined[:,1]

In [17]:
print(data[3000].shape)
print(sum(label))

(135, 289, 3)
97


In [ ]:
train_transform = transforms.Compose([transforms.Resize(255),transforms.RandomResizedCrop(224),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ColorJitter(),
                                transforms.ToTensor()])
test_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

AttributeError: 'list' object has no attribute 'toTensor'